In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Aniridia/GSE204791'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Altered regulation of mRNA and miRNA expression in epithelial and stromal tissue of keratoconus corneas [RNA]"
!Series_summary	"Purpose:  To evaluate conjunctival cell microRNA and mRNA expression in relation to observed phenotype and genotype of aniridia-associated keratopathy (AAK) in a cohort of subjects with congenital aniridia.  Methods:  Using impression cytology, bulbar conjunctival cells were sampled from 20 subjects with congenital aniridia and 20 age and sex-matched healthy control subjects. RNA was extracted and microRNA and mRNA analysis was performed using microarrays. Results were related to the presence and severity of AAK determined by a standardized clinical grading scale and to the genotype (PAX6 mutation?) determined by clinical genetics.   Results:  Of the 2549 microRNAs analyzed, 21 were differentially expressed relative to controls. Among these miR-204-5p, an inhibitor of corneal neovascularization, was downregulated 26.8-fol

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

import numpy as np
nan = np.nan

# Check if gene expression data is available
is_gene_available = True  # Based on the series summary and design, mRNA data is available

# Define the sample characteristics dictionary
sample_characteristics_dict = {
    0: ['age: 59', 'age: 28', 'age: 58', 'age: 56', 'age: 50', 'age: 30', 'age: 53', 'age: 77', 'age: 67', 'age: 29', 'age: 46', 'age: 65', 'age: 81', 'age: 87', 'age: 70', 'age: 79', 'age: 55'],
    1: ['gender: F', 'gender: M'],
    2: ['disease: KC', 'disease: healthy control'],
    3: ['Stage: A4 B4 C3 D4 +', 'Stage: A4 B4 C3 D1 -', 'Stage: A4 B4 C3 D4 ++', 'nan', 'Stage: A2 B4 C1 D3 -', 'Stage: A2 B4 C1 D1 +', 'Stage: A4 B4 C2 D3']
}

# Determine the trait_row, age_row, and gender_row
for key, values in sample_characteristics_dict.items():
    # Filter out NaN values
    filtered_values = [v for v in values if isinstance(v, str)]
    if all('disease' in v for v in filtered_values):
        if len(set(filtered_values)) > 1:
            trait_row = key  # key for Aniridia should be different and not constant
    if all('age' in v for v in filtered_values):
        if len(set(filtered_values)) > 1:
            age_row = key  # key for age
    if all('gender' in v for v in filtered_values):
        if len(set(filtered_values)) > 1:
            gender_row = key  # key for gender

# Define conversion functions
def convert_trait(value):
    val = value.split(":")[1].strip().lower()
    if val == 'kc':
        return 1
    elif val == 'healthy control':
        return 0
    return None

def convert_age(value):
    try:
        return int(value.split(":")[1].strip())
    except ValueError:
        return None

def convert_gender(value):
    val = value.split(":")[1].strip().lower()
    if val == 'f':
        return 0
    elif val == 'm':
        return 1
    return None

# Save cohort information
save_cohort_info('GSE204791', './preprocessed/Aniridia/cohort_info.json', is_gene_available, trait_row is not None)

# If trait data is available, extract clinical feature data
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Aniridia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Aniridia/trait_data/GSE204791.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM6193900': [1, 59, 0], 'GSM6193903': [1, 28, 1], 'GSM6193906': [1, 58, 0], 'GSM6193908': [1, 56, 1], 'GSM6193911': [0, 50, 0], 'GSM6193913': [0, 30, 1], 'GSM6193916': [0, 53, 0], 'GSM6193918': [0, 77, 1], 'GSM6193920': [1, 50, 0], 'GSM6193923': [1, 67, 1], 'GSM6193925': [1, 29, 0], 'GSM6193928': [1, 46, 1], 'GSM6193930': [0, 56, 0], 'GSM6193933': [0, 65, 1], 'GSM6193935': [0, 58, 0], 'GSM6193938': [0, 81, 1], 'GSM6193940': [1, 28, 1], 'GSM6193943': [1, 58, 0], 'GSM6193945': [1, 67, 1], 'GSM6193948': [1, 46, 1], 'GSM6193950': [0, 87, 0], 'GSM6193953': [0, 87, 1], 'GSM6193955': [0, 70, 0], 'GSM6193957': [1, 50, 0], 'GSM6193960': [1, 29, 0], 'GSM6193962': [1, 56, 1], 'GSM6193965': [1, 59, 0], 'GSM6193967': [0, 79, 1], 'GSM6193970': [0, 55, 0], 'GSM6193972': [0, 65, 1], 'GSM6193975': [0, 87, 1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502', 'A_19_P00315506',
       'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315529', 'A_19_P00315541'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the identifier and gene symbol keys
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping) 


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Aniridia/gene_data/GSE204791.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Aniridia')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE204791', './preprocessed/Aniridia/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Aniridia/GSE204791.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Aniridia', the least common label is '0.0' with 15 occurrences. This represents 48.39% of the dataset.
The distribution of the feature 'Aniridia' in this dataset is fine.

Quartiles for 'Age':
  25%: 50.0
  50% (Median): 58.0
  75%: 67.0
Min: 28.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 15 occurrences. This represents 48.39% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

